In [63]:
import pandas as pd
file = 'old.xlsx'
file2 = 'new.xlsx'
desired = 'oldnewfinal.xlsx'

In [64]:
old = pd.read_excel(file)
new = pd.read_excel(file2)
final= pd.read_excel(desired)

In [65]:
old

,Ship_ID1,Account_Zip,Sales,Old_Sales_Person
0,111,customer A_19101,1000,John
1,222,CUstomer A_19101,25,John
2,333,customer B_94516,3000,Sally
3,444,customer B_94516,500,Sally
4,555,customer C_90210,4000,John
5,999,customer x_19101,500000,Robert


In [66]:
new

,Ship_ID1,Account_Zip,Projections,Old_Sales_Person,New_Sales_Person,Territory_ID
0,111,customer A_19101,5000,John,Sally,East
1,xxx,customer A19101,5000,John,Sally,East
2,333,customer B_94516,6000,Sally,John,West
3,444,customer B_94516,6000,Sally,John,West
4,555,customer C_90210,7000,John,John,West
5,666,customer A_19105,5000,John,Sally,East
6,777,customer D_01733,10000,NaN,John,East
7,new_cust,cust x19101,1000000,NaN,John,West


In [67]:
final

,Ship_ID1,Account_Zip,Sales,Projections,Old_Sales_Person,New_Sales_Person,Territory_ID
0,111,customer A_19101,1025,15000,John,Sally,East
1,333,customer B_94516,3500,12000,Sally,John,West
2,555,customer C_90210,4000,7000,John,John,West
3,777,customer D_01733,0,10000,NaN,John,East
4,new_cust,cust x19101,500000,1000000,Rober,John,West


In [68]:
new['Account_corrected'] = new['Account_Zip'].str.lower()
new['Account_corrected'].replace('_','', regex=True,inplace=True)
old['Account_corrected'] = old['Account_Zip'].str.lower()
old['Account_corrected'].replace('_','', regex=True,inplace=True)
new.head()

,Ship_ID1,Account_Zip,Projections,Old_Sales_Person,New_Sales_Person,Territory_ID,Account_corrected
0,111,customer A_19101,5000,John,Sally,East,customer a19101
1,xxx,customer A19101,5000,John,Sally,East,customer a19101
2,333,customer B_94516,6000,Sally,John,West,customer b94516
3,444,customer B_94516,6000,Sally,John,West,customer b94516
4,555,customer C_90210,7000,John,John,West,customer c90210


In [69]:
by_cust = old.groupby(by=['Account_corrected']).agg({'Ship_ID1':'first',
                                               'Account_Zip':'first',
                                               'Sales':'sum',
                                               'Old_Sales_Person':'first'}).reset_index()
by_cust.head()

,Account_corrected,Ship_ID1,Account_Zip,Sales,Old_Sales_Person
0,customer a19101,111,customer A_19101,1025,John
1,customer b94516,333,customer B_94516,3500,Sally
2,customer c90210,555,customer C_90210,4000,John
3,customer x19101,999,customer x_19101,500000,Robert


In [70]:
new_by_cust = new.groupby(by=['Account_corrected']).agg({'Ship_ID1':'first',
                                                   'Account_Zip':'first',
                                                   'Projections':'first',
                                                   'Old_Sales_Person':'first',
                                                   'New_Sales_Person':'first',
                                                   'Territory_ID':'first'}).reset_index()
new_by_cust.head()

,Account_corrected,Ship_ID1,Account_Zip,Projections,Old_Sales_Person,New_Sales_Person,Territory_ID
0,cust x19101,new_cust,cust x19101,1000000,NaN,John,West
1,customer a19101,111,customer A_19101,5000,John,Sally,East
2,customer a19105,666,customer A_19105,5000,John,Sally,East
3,customer b94516,333,customer B_94516,6000,Sally,John,West
4,customer c90210,555,customer C_90210,7000,John,John,West


In [71]:
merged = pd.merge(by_cust, new_by_cust, how='outer', on=['Account_corrected'], indicator='str')
merged

,Account_corrected,Ship_ID1_x,Account_Zip_x,Sales,Old_Sales_Person_x,Ship_ID1_y,Account_Zip_y,Projections,Old_Sales_Person_y,New_Sales_Person,Territory_ID,str
0,customer a19101,111.0,customer A_19101,1025.0,John,111,customer A_19101,5000.0,John,Sally,East,both
1,customer b94516,333.0,customer B_94516,3500.0,Sally,333,customer B_94516,6000.0,Sally,John,West,both
2,customer c90210,555.0,customer C_90210,4000.0,John,555,customer C_90210,7000.0,John,John,West,both
3,customer x19101,999.0,customer x_19101,500000.0,Robert,NaN,NaN,NaN,NaN,NaN,NaN,left_only
4,cust x19101,NaN,NaN,NaN,NaN,new_cust,cust x19101,1000000.0,NaN,John,West,right_only
5,customer a19105,NaN,NaN,NaN,NaN,666,customer A_19105,5000.0,John,Sally,East,right_only
6,customer d01733,NaN,NaN,NaN,NaN,777,customer D_01733,10000.0,NaN,John,East,right_only
